### 一. defer声明

#### 1. defer声明:   
   defer 声明的函数调用, 会被 push 到一个 list 中.   

#### 2. 执行时间  
   这个list 中保存的调用会在包围它的函数 return 后执行
   
#### 3. defer声明通常用来执行 clean-up 操作
defer声明允许我们在打开文件后, 正确的关闭每个文件
```go
func CopyFile(dstName, srcName string) (written int64, err error) {
    src, err := os.Open(srcName)
    if err != nil {
        return
    }
    defer src.Close()

    dst, err := os.Create(dstName)
    if err != nil {
        return
    }
    defer dst.Close()

    return io.Copy(dst, src)
}
```

#### 4. defer声明是可预测的, 有3条准则
1. defer 方法的参数, 在defer函数被声明时计算值    
    * 本例中, defer方法打印"0", 在方法返回后
    ```go
    func a() {
        i := 0
        defer fmt.Println(i)
        i++
        return
    }
    ```
    
2. list 中的 defer 调用, 遵循后进先出(LIFO)的执行顺序  
    * 本例中, 依次打印 "3,2,1,0"  
    ```go
    func b() {
        for i := 0; i < 4; i++ {
            defer fmt.Print(i)
        }
    }
    ```
    
3. defer函数中, 可以读取并分配 returning 函数中的命名返回值  
    * 本例中, defer函数, 在外层函数返回后增加了返回值i的数值
    ```go
    func c() (i int) {
        defer func() { i++ }()
        return 1
    }
    ```

### 二. Panic 
Panic 是内置的方法, 可以阻断一般的控制流并开始 panicing. 
* 当函数 F 调用了 panic() :   
    1. F 的执行流被停止
    2. F 内的 defer 函数被执行
    3. 最后 F 返回到它的 caller 处
    
    
* 对于 F 的 caller :   
    1. 会像 F 似的发生 panic    
    2. 这个过程会从 crash 的点, 持续的向函数调用栈的栈顶进行, 直到当前 goroutine 中的所有方法都返回
    
panic 的动作可以直接调用 panic() 产生, 也可以由运行时 error 产生 (比如数组角标越界)


### 三. Recover

1. recover 可以再产生 panic 的 goroutine 中重新获得控制权.
2. Recover 只在 defer 声明的函数中起到效果:  
   * 效果就是执行 Recover 的函数会 return nil, 并没有其它影响  
   * 如果 goroutine 中产生了 panic , recover 的调用会捕获 panic(val) 中的 val 值, 并继续执行正常处理
   
   
3. 例子  
    * 本例中, 方法 g 有一个参数 i , 如果 i>3 , 发生 panic ; 否则执行 i+1  
      方法 f 中使用 defer 声明了一个执行了 recover 的函数
    ```go
    func TestPanic(t *testing.T) {
        f()
        fmt.Println("Returned normally from f.")
    }

    func f() {
        defer func() {
            if r := recover(); r != nil {
                fmt.Printf("Recovered in f %s\n", r)  // panic中赋予的事字符串
            }
        }()
        fmt.Println("Calling g.")
        g(0)
        fmt.Println("Returned normally from g.")
    }

    func g(i int) {
        if i > 3 {
            fmt.Println("Panicking!")
            panic(fmt.Sprintf("%v", i))
        }
        defer fmt.Println("Defer in g", i)
        fmt.Println("Printing in g", i)
        g(i + 1)
    }
    ```
    
    * 程序输出 
    ```go
    Calling g.
    Printing in g 0
    Printing in g 1
    Printing in g 2
    Printing in g 3
    Panicking!
    Defer in g 3
    Defer in g 2
    Defer in g 1
    Defer in g 0
    Recovered in f 4
    Returned normally from f.
    ```
    * 第二个例子: 发生 recover 的函数会返回 nil 值  
    ```go
    func TestPanic(t *testing.T) {
        f := f()
        fmt.Println("返回", f)
    }

    func f() int {
        defer func() {
            if r := recover(); r != nil {
                fmt.Printf("Recovered in f %s\n", r)  // panic中赋予的事字符串
            }
        }()
        g(1)
        return 99
    }

    func g(i int) {
        panic(fmt.Sprintf("%v", i))

    }

    // output
    Recovered in f 1
    返回 0
    ```